# Semantic Search Through Resumes for HR Tech Startups Using Qdrant

## Utilise Qdrant, Langchain and OpenAI to narrow down on top resumes for the job


In [28]:
import os
import getpass
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import Qdrant
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
loader = PyPDFDirectoryLoader("../data/INFORMATION-TECHNOLOGY")
docs = loader.load()
print(len(docs))


In [11]:
embeddings = OpenAIEmbeddings()

qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [44]:

template = """You are a helpful assistant to a recruiter at a technology firm. You are be provided the following input context \
    from a dataset of resumes of IT professionals. Answer the question based only on the context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
qdrant_retriever = qdrant.as_retriever()
model  = ChatOpenAI()



In [45]:
chain = (
    {"context": qdrant_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [47]:
chain.invoke("Find all resumes which mention Java and HTML?")

'Based on the provided input context, the resumes that mention Java and HTML are:\n\n1. Document 1: The document mentions Java/J2EE technologies and HTML5.\n2. Document 3: The document mentions Java and HTML in the list of programming languages and technologies used.\n\nTherefore, there are two resumes that mention Java and HTML.'